In [1]:
import numpy as np

### training process

- loss_mask
    - 指示哪些token的预测应该被计入损失（例如，在多轮对话中，可能只计算回答部分的损失，而不计算提示部分的损失）。它通常排除了最后一个token，因为最后一个token没有下一个token可供预测。
- logits
    - shape: `[batch_size, seq_len, vocab_size]`
- 对齐 logits 与 labels
    - `labels` (目标) 通常是 `input_ids` 向左移动一位得到 (`input_ids[:, 1:]`)。
    - `shift_logits` 是 logits 去掉最后一个时间步的预测 (`logits[..., :-1, :]`)，以与 labels 对齐。
- 逐 token 计算 loss
    - `nn.CrossEntropyLoss(reduction="none")`

### traing loss

$$
\text{CE}=-\log(P_{\text{true\_token}})
$$

- 假如训练到 0.3 的 loss 水平
    - $P_{\text{true\_token}}=\exp(-0.3)=0.7408$
- 考虑到 qwen vocab size 152064 的水平
    - 瞎猜 $\text{CE}_{\text{rand}}=-\log(\frac1{152064})=11.93$
- 从 PPL（perplexity）的角度
    - $PPL=\exp(\text{CE})$
        - CEL = 0.3 => PPL = 1.35 (能将下一个最可能的词的范围缩小到好像平均只有 1 到 2 个（1.3634）选项一样。)

In [5]:
np.exp(-0.3), np.log(152064), np.exp(0.3), np.exp(11.93)

(0.7408182206817179,
 11.932056763842207,
 1.3498588075760032,
 151751.56167916086)